In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.4f}'.format

# Inputs and Parameters
filename = 'spx_quotedata.csv'

# Black-Scholes European-Options Gamma
def calcGammaEx(S, K, vol, T, r, q, optType, OI):
    if T == 0 or vol == 0:
        return 0

    dp = (np.log(S/K) + (r - q + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    dm = dp - vol*np.sqrt(T) 

    if optType == 'call':
        gamma = np.exp(-q*T) * norm.pdf(dp) / (S * vol * np.sqrt(T))
        return OI * 100 * S * S * 0.01 * gamma 
    else: # Gamma is same for calls and puts. This is just to cross-check
        gamma = K * np.exp(-r*T) * norm.pdf(dm) / (S * S * vol * np.sqrt(T))
        return OI * 100 * S * S * 0.01 * gamma 

def isThirdFriday(d):
    return d.weekday() == 4 and 15 <= d.day <= 21

# Read file
optionsFile = open(filename)
optionsFileData = optionsFile.readlines()
optionsFile.close()

# Get SPX Spot
spotLine = optionsFileData[1].strip()  
spotLine = spotLine.split('Last: ')[1].split(',')[0].strip()
spotPrice = float(spotLine.replace(',', ''))

fromStrike = 0.8 * spotPrice
toStrike = 1.2 * spotPrice

# Get Today's Date
dateLine = optionsFileData[2].strip() 
dateStr = dateLine.split('Date: ')[1].split(' at')[0].strip()
todayDate = datetime.strptime(dateStr, '%B %d, %Y')

# Get SPX Options Data
df = pd.read_csv(filename, sep=",", header=None, skiprows=4)
df.columns = ['ExpirationDate','Calls','CallLastSale','CallNet','CallBid','CallAsk','CallVol',
              'CallIV','CallDelta','CallGamma','CallOpenInt','StrikePrice','Puts','PutLastSale',
              'PutNet','PutBid','PutAsk','PutVol','PutIV','PutDelta','PutGamma','PutOpenInt']

df['ExpirationDate'] = pd.to_datetime(df['ExpirationDate'], format='%a %b %d %Y')
df['ExpirationDate'] = df['ExpirationDate'] + timedelta(hours=16)
df['StrikePrice'] = df['StrikePrice'].astype(float)
df['CallIV'] = df['CallIV'].astype(float)
df['PutIV'] = df['PutIV'].astype(float)
df['CallGamma'] = df['CallGamma'].astype(float)
df['PutGamma'] = df['PutGamma'].astype(float)
df['CallOpenInt'] = df['CallOpenInt'].astype(float)
df['PutOpenInt'] = df['PutOpenInt'].astype(float)

# Calculate Gamma Exposure
df['CallGEX'] = df['CallGamma'] * df['CallOpenInt'] * 100 * spotPrice * spotPrice * 0.01
df['PutGEX'] = df['PutGamma'] * df['PutOpenInt'] * 100 * spotPrice * spotPrice * 0.01 * -1

df['TotalGamma'] = (df.CallGEX + df.PutGEX) / 10**9
dfAgg = df.groupby(['StrikePrice']).agg({
    'TotalGamma': 'sum'
})
strikes = dfAgg.index.values

# Define the range for strikes: 1500 below and 1500 above the SPX spot price
lower_bound = spotPrice - 1500
upper_bound = spotPrice + 1500

# Filter the dfAgg dataframe to only include strikes within the defined range
filtered_dfAgg = dfAgg[(dfAgg.index >= lower_bound) & (dfAgg.index <= upper_bound)]
filtered_strikes = filtered_dfAgg.index.values

# Plotly Chart 1: Absolute Gamma Exposure (filtered to show only strikes within the specified range)
fig1 = go.Figure()

# Add the bar chart for the filtered strikes
fig1.add_trace(go.Bar(
    x=filtered_strikes,
    y=filtered_dfAgg['TotalGamma'],
    name='Gamma Exposure',
    marker=dict(color='blue')
))

# Add a vertical line for the SPX spot price
fig1.add_vline(x=spotPrice, line=dict(color="red"), annotation_text="SPX Spot: {:.0f}".format(spotPrice))

# Update layout with title, labels, and axis range
chartTitle = "Total Gamma: $" + str("{:.2f}".format(df['TotalGamma'].sum())) + " Bn per 1% SPX Move"
fig1.update_layout(
    title=chartTitle,
    xaxis_title='Strike',
    yaxis_title='Spot Gamma Exposure ($ billions/1% move)',
    xaxis=dict(range=[lower_bound, upper_bound]),  # Set x-axis to show the filtered range
    showlegend=True
)

# Show the plot
fig1.show()

# ---=== CALCULATE GAMMA PROFILE ===---
levels = np.linspace(fromStrike, toStrike, 60)

df['daysTillExp'] = [1/262 if (np.busday_count(todayDate.date(), x.date())) == 0 \
                           else np.busday_count(todayDate.date(), x.date())/262 for x in df.ExpirationDate]

nextExpiry = df['ExpirationDate'].min()

df['IsThirdFriday'] = [isThirdFriday(x) for x in df.ExpirationDate]
thirdFridays = df.loc[df['IsThirdFriday'] == True]
nextMonthlyExp = thirdFridays['ExpirationDate'].min()

totalGamma = []
totalGammaExNext = []
totalGammaExFri = []

# For each spot level, calculate gamma exposure at that point
for level in levels:
    df['callGammaEx'] = df.apply(lambda row : calcGammaEx(level, row['StrikePrice'], row['CallIV'], 
                                                          row['daysTillExp'], 0, 0, "call", row['CallOpenInt']), axis = 1)

    df['putGammaEx'] = df.apply(lambda row : calcGammaEx(level, row['StrikePrice'], row['PutIV'], 
                                                         row['daysTillExp'], 0, 0, "put", row['PutOpenInt']), axis = 1)    

    totalGamma.append(df['callGammaEx'].sum() - df['putGammaEx'].sum())

    exNxt = df.loc[df['ExpirationDate'] != nextExpiry]
    totalGammaExNext.append(exNxt['callGammaEx'].sum() - exNxt['putGammaEx'].sum())

    exFri = df.loc[df['ExpirationDate'] != nextMonthlyExp]
    totalGammaExFri.append(exFri['callGammaEx'].sum() - exFri['putGammaEx'].sum())

totalGamma = np.array(totalGamma) / 10**9
totalGammaExNext = np.array(totalGammaExNext) / 10**9
totalGammaExFri = np.array(totalGammaExFri) / 10**9

# ---=== CALCULATE ZERO GAMMA (GAMMA FLIP) ===---
flip_indices = np.where(np.diff(np.sign(totalGamma)))[0]

# Check if there is a gamma flip
if len(flip_indices) > 0:
    # Interpolate to find the exact flip point between the two closest levels
    zeroGamma = (levels[flip_indices[0]] + levels[flip_indices[0] + 1]) / 2
else:
    # If no flip point is found, set it to None or handle accordingly
    zeroGamma = None

# Chart 2: Gamma Exposure Profile
fig2 = go.Figure()

# Adding lines for each Gamma Exposure profile
fig2.add_trace(go.Scatter(
    x=levels,
    y=totalGamma,
    mode='lines',
    name='All Expiries',
    line=dict(color='blue')
))

fig2.add_trace(go.Scatter(
    x=levels,
    y=totalGammaExNext,
    mode='lines',
    name='Ex-Next Expiry',
    line=dict(color='green')
))

fig2.add_trace(go.Scatter(
    x=levels,
    y=totalGammaExFri,
    mode='lines',
    name='Ex-Next Monthly Expiry',
    line=dict(color='orange')
))

# Adding vertical line for SPX Spot price using Scatter for legend
fig2.add_trace(go.Scatter(
    x=[spotPrice, spotPrice],
    y=[min(totalGamma), max(totalGamma)],
    mode='lines',
    name=f"SPX Spot: {spotPrice:,.0f}",
    line=dict(color='red', dash='dash'),
    showlegend=True
))

# Adding vertical line for Gamma Flip using Scatter for legend
if zeroGamma:
    fig2.add_trace(go.Scatter(
        x=[zeroGamma, zeroGamma],
        y=[min(totalGamma), max(totalGamma)],
        mode='lines',
        name=f"Gamma Flip: {zeroGamma:,.0f}",
        line=dict(color='black', dash='dash'),
        showlegend=True
    ))

# Adding horizontal line at y=0
fig2.add_hline(y=0, line_width=1, line_color="grey")

# Chart layout
fig2.update_layout(
    title=f"Gamma Exposure Profile, SPX, {todayDate.strftime('%d %b %Y')}",
    xaxis_title="Index Price",
    yaxis_title="Gamma Exposure ($ billions/1% move)",
    xaxis=dict(range=[fromStrike, toStrike]),
    template="plotly_white"
)

# Adding shading to indicate the regions of positive and negative gamma
fig2.add_shape(type="rect",
               x0=fromStrike, x1=zeroGamma if zeroGamma else toStrike, y0=min(totalGamma), y1=max(totalGamma),
               fillcolor="red", opacity=0.1, line_width=0)

fig2.add_shape(type="rect",
               x0=zeroGamma if zeroGamma else fromStrike, x1=toStrike, y0=min(totalGamma), y1=max(totalGamma),
               fillcolor="green", opacity=0.1, line_width=0)

fig2.show()

# Continue with similar changes for other plots...
